In [2]:
import torch
import torchvision

In [3]:
dets_x1y1 = torch.empty((1000, 2)).normal_(mean=0.25, std=0.1)
dets_wh = torch.empty((1000, 2)).normal_(mean=0.5, std=0.25)
dets_x2y2 = dets_x1y1 + dets_wh
dets_x2y2 = torch.clamp(dets_x2y2, 0, 1)
dets = torch.hstack([dets_x1y1, dets_x2y2])

In [4]:
a = torch.rand((1000, 31))
# dets = torch.rand((1000, 4))
# dets = torch.empty((1000, 4)).normal_(mean=0.5, std=0.25)

a[:, 7] *= 10
dets[0]

tensor([0.2201, 0.2077, 1.0000, 0.4104])

In [5]:
scores = torch.nn.functional.softmax(a, -1)
scores.shape
scores[-1]

tensor([0.0082, 0.0071, 0.0100, 0.0096, 0.0062, 0.0083, 0.0103, 0.7801, 0.0070,
        0.0089, 0.0056, 0.0050, 0.0049, 0.0095, 0.0071, 0.0052, 0.0100, 0.0051,
        0.0058, 0.0063, 0.0054, 0.0088, 0.0114, 0.0096, 0.0054, 0.0047, 0.0071,
        0.0078, 0.0057, 0.0062, 0.0074])

In [6]:
start_index = 1
conf_thresh = 0.2
max_feats = 50

In [7]:
obj_scores, obj_ids = torch.max(scores[:, start_index:], 1)  # находим максимальные скоры в каждом боксе из 1000 и получаем соотв id классов
print(obj_ids[:20])
keep = torchvision.ops.batched_nms(dets, obj_scores, obj_ids, 0.5)  # фильтруем боксы, keep возвращает номера боксов, которые нужно оставить
print(keep[:20])

keeped_scores = obj_scores[keep]
keeped_dets = dets[keep]
keeped_obj_ids = obj_ids[keep]

sorted_scores, sorted_indices = torch.sort(keeped_scores, descending=True)
sorted_ids_by_score = keeped_obj_ids[sorted_indices]
sorted_dets_by_score = keeped_dets[sorted_indices]
# print(sorted_scores)

res_scores = sorted_scores[sorted_scores > conf_thresh][:max_feats]
res_obj_ids = sorted_ids_by_score[sorted_scores > conf_thresh][:max_feats]
res_dets = sorted_dets_by_score[sorted_scores > conf_thresh][:max_feats]
print(res_obj_ids)

# # max_conf = torch.zeros(scores.shape[0]).to(cur_device)  # 1000
(obj_ids[sorted_indices[0]], sorted_ids_by_score[0], keep_ids[0])

tensor([ 6,  6, 26,  6,  6,  6,  6,  6,  6, 17,  6,  6,  6,  6,  6,  6,  6,  6,
         6,  6])
tensor([396,  62, 914, 556, 605, 949, 284, 908, 880, 390, 578,  20,  70,  75,
        974, 393, 822, 784, 748, 301])
tensor([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
        6, 6])


NameError: name 'keep_ids' is not defined

In [8]:
a = torch.tensor([0,1,2,3,4,5,6,7])
mask = (a > 3)
mask = mask[:2]
a[mask]

IndexError: The shape of the mask [2] at index 0 does not match the shape of the indexed tensor [8] at index 0

In [9]:
import json

with open('visual_genome_categories.json') as f:
    cats = json.load(f)
    cats = cats['categories']
    
id2cat = {el['id']: el['name'] for el in cats}

In [10]:
id2cat[1]

'goal'

In [11]:
a = torch.Tensor([6,5,4,3,2])
b = a > 3
b.sum().item()

3

In [12]:
b[:4] = True
b

tensor([ True,  True,  True,  True, False])

In [17]:
z = torch.rand((1, 150, 2048))
z[:, 50:] = torch.zeros((2048))
z

tensor([[[0.2177, 0.2702, 0.5557,  ..., 0.0906, 0.6410, 0.4039],
         [0.3015, 0.3306, 0.2530,  ..., 0.7917, 0.2975, 0.0917],
         [0.0112, 0.2989, 0.2750,  ..., 0.6124, 0.5619, 0.9130],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]])

In [24]:
(z[0].sum(dim=1) != 0).sum()

tensor(50)

In [29]:
torch.eye(5)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.]])